<a href="https://colab.research.google.com/github/samibahig/recover/blob/master/DecisionTreeProteomicThibault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### Pour Thibault, il y a rien ici, c'est la première partie du code qui nettoie les données et les met en view
"""
This script extract a data and metadata from the files
it gives dataframes and a labels file
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
data_path = ''

#print('\nMETADATA :')
metadata_filename = data_path + '/content/metadata.csv'
meta_df = pd.read_csv(metadata_filename)
#print(meta_df.columns)
meta_df.columns = ['#', 'plate', '-', 'symptoms'] + list(meta_df)[4:]
#print(meta_df.columns)
#print('available metadata :', list(meta_df))
meta_idx = meta_df['ID'].to_list()
meta_label = meta_df['symptoms'].to_list()
#print('------------------')
#print(list(zip(meta_idx, meta_label)))
#print('------------------')
meta_id_label_dict = {str(k): 1 if v=='S' else 0 for k, v in zip(meta_idx, meta_label)}

#DF1 : proteomics
#print('\nPROEOMICS DATA :')
proteomics_data_filename = '/content/proteomics.csv'

dim_df = pd.read_csv(proteomics_data_filename, nrows=1)
#print('--------')
#print(dim_df)
#print('--------')
dim = len(list(dim_df))
#print(dim)
#print('------------')
print('# of columns in source csv file :', dim)
all_cols = [i for i in range(dim)]

print('--------')
feat_cols = all_cols[1:-4]
print(feat_cols)
print('--------')
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])

idx_df = pd.read_csv(proteomics_data_filename, skiprows=6, index_col=0, skipfooter=4, usecols=[0], engine='python')
idx = list(idx_df.index.values)
print('# of idx : ', len(idx))
print('first id :', idx[0])
print('last id :', idx[-1])

df1 = pd.read_csv(proteomics_data_filename, skiprows=6, dtype=np.float32, skipfooter=4, usecols=feat_cols, engine='python')
assert df1.shape[0] == len(idx)
assert df1.shape[1] == len(features)

df1['idx'] = idx
df1.set_index('idx', inplace=True)
df1.columns = features
print('# of Nan values :', df1.isna().sum().sum())

#clean data of samples that are not in metadata :
idx = df1.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df1['label'] = y
df1 = df1[df1.label != 'to_remove']

#create X and y matrices for ML :
y = list(df1['label'])
del df1['label']
print('---------')
print(df1)
print('---------')
X = df1.to_numpy()
print('proteomics data :')
print('# of samples : ', df1.shape[0])
print('# of features : ', df1.shape[1])
print('labels:', list(dict.fromkeys(y)))


## save X and y in pickles if you want :
##data_name = 'recover_multiomics_'
##feat_dict = {k: str(v) for k, v in zip(range(len(list(df))), list(df))}
##with open(data_path + data_name + 'feat_dict', 'wb') as fo:
##    pkl.dump(feat_dict, fo)
##with open(data_path + data_name + 'X', 'wb') as fo:df
##            pkl.dump(X, fo)
##with open(data_path + data_name + 'y', 'wb') as fo:
##            pkl.dump(y, fo)

# of columns in source csv file : 189
--------
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]
--------
# of features :  184
first feature : Q96SB3
last feature : P09603
# of idx :  104
first id : 5-139
last id : 3-043
# of Nan values 

In [5]:
### Pour Thibault, j'utilise la méthode RandomGrid poru trouver mes meilleurs paramètres (ici j'ai trouvé ce set de parametres sur Internet)
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
random_grid = {'max_depth': [None, 2, 5, 10],
               'min_samples_leaf': [1, 5, 10],
               'min_samples_split': [2, 10, 20],
               "criterion": ["gini", "entropy"],
               "max_leaf_nodes": [None, 5, 10, 20]}
pprint(random_grid)

{'criterion': ['gini', 'entropy'],
 'max_depth': [None, 2, 5, 10],
 'max_leaf_nodes': [None, 5, 10, 20],
 'min_samples_leaf': [1, 5, 10],
 'min_samples_split': [2, 10, 20]}


In [34]:
### Pour Thibault, ici je teste mes différents paramètres pour choisir les meilleurs, puis après les meilleurs sont dans la prochaine cellule 
from sklearn.tree import DecisionTreeClassifier
# Use the random grid to search for best hyperparameters
# First create the base model to tune
tree_clf = DecisionTreeClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
tree_clf = RandomizedSearchCV(estimator = tree_clf, param_distributions = random_grid, n_iter = 200, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
tree_clf.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    5.7s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
          

In [10]:
### Pour Thibault, ici soit je fais mon train-test standard 80/20, puis j'entraine le modèle et calcule l'accuracy, puis après j'essaie le Nootstarpping a la prochaien cellule.  
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
tree_clf = DecisionTreeClassifier(criterion='gini', max_depth= 10, max_leaf_nodes= 10, min_samples_leaf = 1, min_samples_split = 10)
tree_clf.fit(X_train, y_train)
predictions = tree_clf.predict(X_test, check_input=True)
#print(len(predictions), predictions)
#(predictions).astype(int).sum()/len(y_test)
accuracy_score(y_test, predictions)

0.7

In [33]:
###  Boostraping , Pour Thibault, tu peux essayer ici, mais a chaque fosi que je roule cette cellule, j'obtiens un résultat différent.
# Creating empty list to hold accuracy values
AccuracyValues=[]
n_times=30

## bootstrapping
from sklearn import metrics
for i in range(n_times):
    #Split the data into training and testing set
    from sklearn.model_selection import train_test_split
    # Changing the seed value for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+i)
    clf = DecisionTreeClassifier(criterion='gini', max_depth= 10, max_leaf_nodes= 10, min_samples_leaf = 1, min_samples_split = 10)
    clf.fit(X_train, y_train)
    ########################################################
    predictions = clf.predict(X_test, check_input=True)
    Accuracy=metrics.accuracy_score(y_test, predictions)
    AccuracyValues.append(np.round(Accuracy))

# Result of all bootstrapping trials
print(y_test, predictions)
print(AccuracyValues)
 
# Final accuracy
print('Final average accuracy',np.mean(AccuracyValues))

[1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] [1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0]
[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]
Final average accuracy 0.4
